# Домашнее задание № 1 по курсу "Обнаружение и распознавание сигналов"

### Мусина Камила, ИУ8-82

#### Выбранные 3 интереса:
1. Разработка на С/С++ (Программирование)
2. Кинематограф
3. Изучение немецкого языка

#### Признаки аккаунтов:
1. День рождения
2. Пол
3. Количество друзей
4. Количество подписок на группы
5. Номер мобильного телефона
6. Количество фотографий
7. Семейное положение
8. Количество подписок на публичные страницы
9. Информация о внешних ссылках (указана/отсутствует)
10. ID пользователя
11. Количество подписчиков
12. Информация о месте обучения (указана/отсутствует)
13. Информация о месте работы (указана/отсутствует)
14. Дата последней сессии в вк
15. Политические предпочтения 
    1 — коммунистические
    2 — социалистические
    3 — умеренные
    4 — либеральные
    5 — консервативные
    6 — монархические
    7 — ультраконсервативные
    8 — индифферентные
    9 — либертарианские
16. Главное в людях
    1 — ум и креативность
    2 — доброта и честность
    3 — красота и здоровье
    4 — власть и богатство
    5 — смелость и упорство
    6 — юмор и жизнелюбие
17. Главное в жизни
    1 — семья и дети
    2 — карьера и деньги
    3 — развлечения и отдых
    4 — наука и исследования
    5 — совершенствование мира
    6 — саморазвитие
    7 — красота и искусство
    8 — слава и влияние
18. Количество добавленных фотоальбомов
19. Количество добавленной музыки
20. Количество добавленных видео

In [23]:
import vk
import csv
import requests
import time

token = 'token'
group_list = ['cpp_lib', 'cinemakinoart', 'ldeutsch']
all_users = []

## Сбор ID пользователей по интересам

In [24]:
def get_users(group_id):
    good_id_list = []
    offset = 0
    response = requests.get('https://api.vk.com/method/groups.getMembers', params={
        'access_token':token,
        'v':5.103,
        'group_id': group_id,
        'sort':'id_desc',
        'offset':offset,
        'fields':'last_seen'
    }).json()['response']
    for item in response['items']:
        try:
            if item['last_seen']['time'] >= 1677500000 and item['is_closed'] is False:
                good_id_list.append(item['id'])
        except Exception as E:
            continue
    return good_id_list

## Парсер по страницам пользователей ВК

In [25]:
def parse_id(ids, group):
    parsed_data = []
    info = vk_api.users.get(user_id=ids, fields='''sex, bdate, relation, career, counters, personal, contacts, connections, last_seen''')
  
    i = 0
    for id in info:
        result = []
        result.append(group)
        result.append(id[u'id'])
        result.append(id[u'sex'])
        result.append(id[u'last_seen'][u'time'])
        result.append(1 if  u'university' in id else 0)
        if u'counters' in id:
            result.append(id[u'counters'][u'friends'])
            result.append(id[u'counters'][u'audios'] if u'audios' in u'counters' else 0)
            result.append(id[u'counters'][u'photos'])
            result.append(id[u'counters'][u'groups'])
            result.append(id[u'counters'][u'followers'])
            result.append(id[u'counters'][u'pages'] if u'pages' in u'counters' else 0)
            result.append(id[u'counters'][u'albums'] if u'albums' in u'counters' else 0)
        else:
            for i in range(6):
                result.append(0)
        result.append(id[u'bdate'] if  u'bdate' in id  else 0)
        result.append(id[u'relation'] if u'relation' in id else 0)
        result.append(1 if  u'career' in id else 0)
        if u'personal' in id:
            result.append(id[u'personal'][u'political'] if u'political' in id else 0)
            result.append(id[u'personal'][u'people_main'] if u'people_main' in id else 0)
            result.append(id[u'personal'][u'life_main'] if u'life_main' in id else 0)
        else:
            for i in range(3):
                result.append(0)
        result.append(1 if  u'mobile_phone' in id else 0)
        result.append(1 if  u'connections' in id else 0)
        parsed_data.append(result)
        
    return parsed_data

In [28]:
vk_api = vk.API(access_token=token, v='11.9.9')

for group in group_list:
    try:
        users = get_users(group)
        parsed_users = parse_id(users, 1 if group == 'cpp_lib' else (2 if group == 'cinemakinoart' else 3))
        for i in range(len(parsed_users)):
            all_users.append(parsed_users[i])
        time.sleep(1)
    except KeyError as E:
        print(group, E)
        continue


## Создание csv-датасета

In [29]:
with open('vk_dataset.csv', 'w', newline = '') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
    quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Interest', 'ID', 'Sex', 'Last Seen', 'University', 'Friends',
                         'Audios', 'Photos', 'Groups', 'Followers', 'Pages', 'Albums',
                         'Birthday', 'Relation Status', 'Career', 'Political View', 
                         'People Main', 'Life Main', 'Mobile Phone', 'Connections', 'Video'])
    for user in all_users:
        filewriter.writerows(all_users)